# Домашнее задание к лекции "Основы веб-скрапинга и работы с API"

### Задание 1

Будем парсить страницу со свежеми новостям на habr.com/ru/all/.

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

KEYWORDS = ['python', 'парсинг']

Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы).

В итоге должен формироваться датафрейм вида: <дата> - <заголовок> - <ссылка>

In [1]:
# импортируем необходимые для работы библиотеки
import time
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json
from pprint import pprint

In [7]:
# определяем список ключевых слов, которые нам интересны
KEYWORDS = ['python', 'парсинг', 'алгоритм']

In [8]:
# получаем страницу с самыми свежими постами
req = requests.get('https://habr.com/ru/all/')

# формируем soup-объект с данными из html
soup = BeautifulSoup(req.text, 'html.parser')

In [9]:
# извлекаем посты (во всем soup-объекте ищем нужный тег article, возвращается список всех вхождений тега в нужный параметр)
posts = soup.find_all('article', class_='post post_preview')
# posts

In [10]:
habr_news = pd.DataFrame()

for post in posts:
    # делаем задержку исполнения
    time.sleep(0.2)
    post_id = post.parent.attrs.get('id')
   
    # если идентификатор не найден, это что-то странное, пропускаем
    if not post_id:
        continue
        
    post_id = int(post_id.split('_')[-1])
    
    if post.find(class_='post__text'): 
        article = post.find(class_='post__text').text
    else:
        continue
        
    # приводим содержимое статей к нижнему регистру    
    article_lower = article.lower()
    
   # ищем вхождение хотя бы одного желаемого слова
    for keyword in KEYWORDS:
        if keyword in article_lower:
            time_element = post.find('span', class_='post__time')
            title_element = post.find('a', class_='post__title_link')
            row = {'date': time_element.text, 
                   'title': title_element.text, 
                   'url': title_element.attrs.get('href'), 
                   "keyword": keyword}
            
            # делаем объединение таблиц с помощью функции pd.concat()
            habr_news = pd.concat([habr_news, pd.DataFrame([row])], ignore_index=True)  
habr_news

,date,title,url,keyword
0,сегодня в 14:27,Новый алгоритм проверки пересечений в графах п...,https://habr.com/ru/post/523086/,алгоритм


### Задание 2

Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса Avast Hack Ckeck. Список email-ов задаем переменной в начале кода:
EMAIL = [xxx@x.ru, yyy@y.com]

В итоге должен формироваться датафрейм со столбцами: <почта> - <дата утечки> - <источник утечки> - <описание утечки>

Подсказка: сервис работает при помощи "скрытого" API. Внимательно изучите post-запросы.

In [11]:
email = ['xxx@x.ru', 'yyy@y.com']
def get_email_leaks(email):
    URL = 'https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data'
    data = {
        'emailAddresses': email
    }
    data = json.dumps(data)

    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.75 Safari/537.36',
               'Vaar-Header-App-Product': 'hackcheck-web-avast',
                'Vaar-Version': '0'}
    req = requests.post(URL, data=data, verify=False,  headers=headers)
    return json.loads(req.text)

pprint(get_email_leaks(email))

C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:979: InsecureRequestWarning: Unverified HTTPS request is being made to host 'identityprotection.avast.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


{'breaches': {'12': {'description': 'Popular Russian social networking '
                                    'platform VKontakte was breached in late '
                                    '2012. Over 100 million clear-text '
                                    'passwords were compromised in the breach. '
                                    'Breached credential sets included '
                                    "victims' e-mail addresses, passwords, "
                                    'dates of birth, phone numbers and '
                                    'location details. The credential set was '
                                    'advertised on a dark web marketplace as '
                                    'of June 2016 for a price of one bitcoin. ',
                     'publishDate': '2016-10-29T00:00:00Z',
                     'recordsCount': 91262655,
                     'site': 'vk.com'},
              '13094': {'description': 'In October 2017, a customer database '
     

In [13]:
# Для того, чтобы сформировать нужным нам датафрейм, нам нужно объединить датафреймы breaches и data (словари с вложенными словарями, добираемся до нужной информации)
breaches = get_email_leaks(email)['breaches']
data = get_email_leaks(email)['data']

breaches_df = pd.DataFrame()
data_df = pd.DataFrame()

# проходимся циклом по словарю breaches
for key, value in breaches.items():
    
    # делаем задержку исполнения
    time.sleep(0.2)
    
    row_breaches = {'description': value['description'], 'source': value['site'],
                    'date': pd.to_datetime(value['publishDate'])}
    
    breaches_df = pd.concat([breaches_df, pd.DataFrame([row_breaches])])
    
# проходимся циклом по словарю data    
for key, value in data.items():
    for k, v in value.items():
        row_data = {'e-mail': k, 'source': key}
        data_df = pd.concat([data_df, pd.DataFrame([row_data])])

email_leaks_df = df_data.merge(df_breaches, on=['source'], how='left')
email_leaks_df.head()

C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:979: InsecureRequestWarning: Unverified HTTPS request is being made to host 'identityprotection.avast.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:979: InsecureRequestWarning: Unverified HTTPS request is being made to host 'identityprotection.avast.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


,e-mail,source,description,date
0,xxx@x.ru,parapa.mail.ru,"In July and August 2016, two criminals execute...",2017-02-14 00:00:00+00:00
1,xxx@x.ru,verifications.io,Big data e-mail verification platform verifica...,2019-03-28 00:00:00+00:00
2,xxx@x.ru,adobe.com,"In October of 2013, criminals penetrated Adobe...",2016-10-21 00:00:00+00:00
3,yyy@y.com,adobe.com,"In October of 2013, criminals penetrated Adobe...",2016-10-21 00:00:00+00:00
4,xxx@x.ru,cdprojektred.com,"In March 2016, CDProjektRed.com.com's forum da...",2017-01-31 00:00:00+00:00
